# Data processing


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!pip install wandb -q
output_dir = "./"
import wandb
wandb.login()

wandb: Currently logged in as: asif-dev-bd. Use `wandb login --relogin` to force relogin


True

In [4]:
!pip install nltk -q
!pip install transformers[torch] -q
!pip install rouge_score -q
!pip install evaluate -q
!pip install rouge_score -q
!pip install jiwer -q
!pip install sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.4 MB/s eta 0:00:00


In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# token = user_secrets.get_secret("HF_TOKEN")

In [7]:
import json
import os
import time
import random
import re
import string
import time
import numpy as np
import pandas as pd
import torch
import jiwer
from datasets import Dataset, load_metric
from evaluate import load
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.model_selection import train_test_split
from transformers import (AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, AutoTokenizer, BartTokenizer,EarlyStoppingCallback,
                          DataCollatorForSeq2Seq, MBart50Tokenizer, Trainer, Seq2SeqTrainer,
                          TrainingArguments,GenerationConfig, pipeline)
from transformers.trainer_utils import get_last_checkpoint


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def replace_with_homophones(word):
    adjacent_keys = {
            "অ": "আও",
            "আ": "অও",
            "ই": "ঈউই",
            "ঈ": "ইঈ",
            "উ": "ঊউই",
            "ঊ": "উঊ",
            "ঋ": "ঋ",
            "এ": "ঐএই",
            "ঐ": "এঐই",
            "ও": "ঔঅও",
            "ঔ": "ওঔ",
            "ক": "খগ",
            "খ": "কগ",
            "গ": "ঘগ্",
            "ঘ": "গগ্",
            "ঙ": "ঙং",
            "চ": "ছজ",
            "ছ": "চজ",
            "জ": "ঝয",
            "ঝ": "জয",
            "ঞ": "ঞম",
            "ট": "ঠড",
            "ঠ": "টডথ",
            "ড": "ঢদধ",
            "ঢ": "ডদধ",
            "ণ": "ণনম",
            "ত": "থদত",
            "থ": "তদদ্",
            "দ": "ধড",
            "ধ": "দড",
            "ন": "ণম",
            "প": "ফব",
            "ফ": "প",
            "ব": "ভব্",
            "ভ": "ব",
            "ম": "মন",
            "য": "জঝ",
            "র": "লর্যড়ঢ়য়",
            "ল": "রল",
            "শ": "সষ",
            "ষ": "শস",
            "স": "শষ",
            "হ": "হ্",
            "ড়": "ঢ়য়র",
            "ঢ়": "ড়য়র",
            "য়": "ড়ঢ়্",
            "ৎ": "ৎ্তট",
            "ং": "ঙ্",
            "ঃ": "ঃ্",
            "ঁ": "ঁ্",
        }
    diacritic_mapping = {

        "া": "িীুূৃেৈোৌ",
        "ি": "ীাুূ",
        "ী": "িাুূ",
        "ু": "ূিীা",
        "ূ": "ুিীা",
        "ৃ": "েৈা",
        "ে": "ৈৃো",
        "ৈ": "েৃো",
        "ো": "ৌেৈা",
        "ৌ": "োেৈা",
    }

    idx = random.randint(0, len(word) - 1)
    char = word[idx]

    if char in adjacent_keys:
        word = word[:idx] + random.choice(adjacent_keys[char]) + word[idx + 1:]
        return word
    elif char in diacritic_mapping:
        new_diacritic = random.choice(diacritic_mapping[char])
        word = word[:idx] + new_diacritic + word[idx + 1:]
    return word


In [12]:
def swap_adjacent_chars(word):
    if len(word) < 2:
        return word
    idx = random.randint(0, len(word) - 2)
    return word[:idx] + word[idx + 1] + word[idx] + word[idx + 2:]

def remove_char(word):
    if len(word) < 2:
        return word
    idx = random.randint(0, len(word) - 1)
    return word[:idx] + word[idx + 1:]

def insert_char(word):
    idx = random.randint(0, len(word))
    char = random.choice(string.ascii_lowercase)
    return word[:idx] + char + word[idx:]

def combine_words(words):
    idx = random.randint(0, len(words) - 2)
    words[idx] = words[idx] + words[idx + 1]
    del words[idx + 1]
    return words

def transpose_char(word):
    if len(word) < 2:
        return word
    idx = random.randint(0, len(word) - 2)
    word = word[:idx] + word[idx + 1] + word[idx] + word[idx + 2:]
    return word

def repeat_char(word):
    if len(word) < 1:
        return word
    idx = random.randint(0, len(word) - 1)
    word = word[:idx] + word[idx] + word[idx] + word[idx + 1:]
    return word

def remove_diacritic(word):
    diacritics = "ািীুূৃেৈোৌ"
    new_word = ''
    for char in word:
        if char in diacritics and random.random() < 0.5:
            continue
        new_word += char
    return new_word if new_word else word

def replace_wrong_diacritic(word):
    wrong_diacritic = {
        "া": "ে",
        "ি": "ী",
        "ী": "ি",
        "ু": "ূ",
        "ূ": "ু",
    }
    new_word = ''
    for char in word:
        if char in wrong_diacritic and random.random() < 0.5:
            new_word += wrong_diacritic[char]
        else:
            new_word += char
    return new_word


In [ ]:

def modify_word_based_on_error_type(word, error_type):
    if error_type == "swap":
        return swap_adjacent_chars(word)
    elif error_type == "remove":
        return remove_char(word)
    elif error_type == "insert":
        return insert_char(word)
    elif error_type == "adjacent":
        return replace_with_homophones(word)
    elif error_type == "combine":
        return word
    elif error_type == "transpose":
        return transpose_char(word)
    elif error_type == "repeat":
        return repeat_char(word)
    elif error_type == "remove_diacritic":
        return remove_diacritic(word)
    elif error_type == "replace_wrong_diacritic":
        return replace_wrong_diacritic(word)



In [ ]:
def introduce_errors(query, error_rate):
    words = query.split()
    if len(words) == 0:
        return query
    num_errors = random.randint(0, 1) #int((len(words) - 1) * (error_rate - 0.5)))
    for _ in range(num_errors):
        if random.random() < error_rate:
            idx = random.randint(0, len(words) - 1)
            error_types = ["swap", "remove", "insert", "adjacent", "combine",
                           "transpose", "repeat", "remove_diacritic", "replace_wrong_diacritic"]
            error_type = random.choice(error_types)
            if error_type == "combine" and len(words) > 1:
                words = combine_words(words)
            else:
                words[idx] = modify_word_based_on_error_type(words[idx], error_type)
    return " ".join(words)

In [ ]:
def generate_dataset(input_file, target_file, error_rate):
    pairs = []
    for i, t in zip(input_file, target_file):
        if not i.strip() and t.strip():
            continue
        erroneous_query = introduce_errors(i, error_rate)
        pairs.append((erroneous_query, t))
    return pairs

def extract(data,dataset_type, column):
    target_data = data[column] #target,input
    target_data = target_data.dropna()
    def replace_multiple_digits(text):
        return re.sub(r"(\d)\1+", r"\1", text)
    cleaned_target_data = target_data.apply(replace_multiple_digits)
    cleaned_target_list = cleaned_target_data.tolist()
    json_data = json.dumps(cleaned_target_list, ensure_ascii=False)
    with open(f"{dataset_type}_{column}.json", "w", encoding="utf-8") as f:
        f.write(json_data)

def get_json(path):
    with open(path, "r", encoding="utf-8") as f:
        json_data = json.load(f)
        return json_data

def create_final_dataset(path, dataset_type, error_rate): #path of the original csv data
    data = pd.read_csv(path)
    print(data.shape)
    extract(data, dataset_type, 'Input')
    extract(data, dataset_type, 'Target')
    input_file = get_json(f"{output_dir}{dataset_type}_Input.json")
    target_file = get_json(f"{output_dir}{dataset_type}_Target.json")
    dataset=generate_dataset(input_file, target_file, error_rate)
    with open(f"{dataset_type}_data.json", "w", encoding="utf-8") as f:
        json.dump(dataset, f, ensure_ascii=False)
    return dataset

# New Section

In [13]:
    test_raw_path ="/content/drive/MyDrive/Colab Notebooks/mBART 50/test.csv"
    train_raw_path = "/content/drive/MyDrive/Colab Notebooks/mBART 50/train.csv"
    val_pairs = create_final_dataset(test_raw_path,"eval", error_rate=0.3)
    train_pairs = create_final_dataset(train_raw_path,"train", error_rate=0.3)
    print(len(train_pairs))
    print(train_pairs[:5])

(900, 2)
(1356300, 2)
1356300
[('রিয়াল ভাবটে পরে তারা আদলে 1 পয়েন্টে এগিয়ে ।', 'রিয়াল ভাবতে পারে তারা আসলে 1 পয়েন্টে এগিয়ে ।'), ('পড়ে সেখানে সংক্ষিপ্ত সনবেশ অনুষতিিত হয় ।', 'পরে সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত হয় ।'), ('বেঁচে থাকার কথা ভাবিনি বলচিলেন নূর মোযামদ ।', 'বেঁচে থাকার কথা ভাবিনি বলছিলেন নূর মোহাম্মদ ।'), ('টিনি চুয়াদাংগা ষহতের মল্লিকপাড়া এলাকার সাইফুল উসলামেড় সট্ড়ী ।', 'তিনি চুয়াডাঙ্গা শহরের মল্লিকপাড়া এলাকার সাইফুল ইসলামের স্ত্রী ।'), ('তাঁক ধুনট িপজল সাস্থ্য কমপলেক্সে আনা গল করটব্যরত চিকতআক তাঁকে মৃত বকে ঘোষণা করেন ।', 'তাঁকে ধুনট উপজেলা স্বাস্থ্য কমপ্লেক্সে আনা হলে কর্তব্যরত চিকিৎসক তাঁকে মৃত বলে ঘোষণা করেন ।')]


In [14]:
train, val = train_test_split(train_pairs, test_size=0.005, random_state=42)

In [15]:
print(len(train))
print(len(val))
print(train[:5])
print(val[:5])

1349518
6782
[('এতে চততগ্রাম বিভাগের nবভিহ্ন জেলার যামণদড় এনং চততগ্রামেড় মুকতজোদ্ধারা উপস্থিত ছিলেন ।', 'এতে চট্টগ্রাম বিভাগের বিভিন্ন জেলার কামান্ডার এবং চট্টগ্রামের মুক্তিযোদ্ধারা উপস্থিত ছিলেন ।'), ('কিণতু এর বিওরীট পরতিক্রিয়াও হরে পারে ।', 'কিন্তু এর বিপরীত প্রতিক্রিয়াও হতে পারে ।'), ('মুসলমান সবখাাগরিষ্ঠ দেশগুলোর সংগঠন ওআইসি ে উদাোগে সাদারণ পরিষদ ওকটি পরস্তান উততপন করা হয় ।', 'মুসলমান সংখ্যাগরিষ্ঠ দেশগুলোর সংগঠন ওআইসির উদ্যোগে সাধারণ পরিষদে একটি প্রস্তাব উত্থাপন করা হয় ।'), ('তবে পিডিবির সূটর নলেছে বিদাুর উৎপদন বাবহৃত গ্যাসের ডাম বাড়ানো হয়ছে ।', 'তবে পিডিবির সূত্র বলেছে বিদ্যুৎ উৎপাদনে ব্যবহৃত গ্যাসের দাম বাড়ানো হয়েছে ।'), ('এর মধােই ছড়িয়ে ছিটিয়ে একট প্রতিষ্ঠানের এযেকটি করণার ।', 'এর মধ্যেই ছড়িয়ে ছিটিয়ে একটি প্রতিষ্ঠানের একেকটি কর্নার ।')]
[('তিনি বলেন আনরা 1 ভাগ অপহরণের ঘটনার রযসা উদ্\u200cঘাটন ও বিকতিমকে উদ্ধার করতে পেরেছি ।', 'তিনি বলেন আমরা 1 ভাগ অপহরণের ঘটনার রহস্য উদ্\u200cঘাটন ও ভিকটিমকে উদ্ধার করতে পেরেছি ।'), ('সপ্ড়তি বিশরিংকলা অদক্কতা সর দুরণতির অভিযোগে লাবাদির

# Tokenization

In [16]:
class CreateDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_length=20):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text, target_text = self.data[idx]
        input_encodings = self.tokenizer(input_text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)

        target_encodings = self.tokenizer(target_text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)

        return {"input_ids": input_encodings['input_ids'].squeeze(),
                "attention_mask": input_encodings["attention_mask"].squeeze(),
                "labels": target_encodings['input_ids'].squeeze()}

# Evaluation Metrics

In [17]:
cer = load("cer")
wer = load("wer")
metric = load("sacrebleu")
meteor = load('meteor')
rouge = load('rouge')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [18]:
output_dir = "/content/drive/MyDrive/Colab Notebooks/mBART 50/Models"

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def setup(train_pairs,val_pairs, model, tokenizer, max_token_length=20, sample_size=1000, batch_size=100,  epochs=1,  ): #(args):


    train_data = CreateDataset(train_pairs[:sample_size],
                                           tokenizer, max_length=max_token_length) #args.max_length)
    val_data   = CreateDataset(val_pairs[:int(sample_size*.01)]  , tokenizer, max_length=max_token_length) #args.max_length)

    print(f"Number of parameters: {model.num_parameters():,}")

    seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [[label.strip()] for label in labels]

        return preds, labels

    def compute_metrics(eval_preds):
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]

        preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        idx = random.randint(0, len(decoded_preds))
        print(f"Prediction: {decoded_preds[idx]}")
        print(f"Label: {decoded_labels[idx]}")

        result = {}

        _rouge = rouge.compute(predictions=decoded_preds, references=decoded_labels)
        result["rouge1"] = _rouge["rouge1"]
        result["rouge2"] = _rouge["rouge2"]
        _wer = jiwer.wer(decoded_preds, decoded_labels)
        result["wer"] = _wer
        _cer = jiwer.cer(decoded_preds, decoded_labels)
        result["cer"] = _cer

        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

        _bleu = metric.compute(predictions=decoded_preds, references=decoded_labels)
        result["bleu"] = _bleu["score"]/100
        _meteor = meteor.compute(predictions=decoded_preds, references=decoded_labels)
        result["meteor"] = _meteor["meteor"]
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)

        result = {k: round(v, 3) for k, v in result.items()}
        return result

    training_args = Seq2SeqTrainingArguments(
        output_dir= output_dir,#args.output_dir,
        num_train_epochs=epochs,#args.num_epochs,
        per_device_train_batch_size= batch_size, #args.batch_size,
        per_device_eval_batch_size=batch_size, #args.eval_batch_size,
        gradient_accumulation_steps=2, #args.gradient_accumulation_steps,
        logging_dir=output_dir, #args.logging_dir,
        logging_steps=10, #args.logging_steps,
        evaluation_strategy="steps",
        eval_steps = 500,
        save_steps=500,#args.save_steps,
        save_total_limit=1,
        predict_with_generate=True,
        load_best_model_at_end=True,
        disable_tqdm=False,
        learning_rate=1e-5,#args.lr,
        lr_scheduler_type='constant',#args.lr_scheduler_type,
#         torch_compile=True, #args.torch_compile,
        optim='adamw_torch_fused',#args.optimizer,
        metric_for_best_model="bleu",
        report_to="wandb",
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        data_collator=seq2seq_data_collator,
        train_dataset=train_data,
        eval_dataset=val_data,
        compute_metrics=compute_metrics,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=50)]
    )
    return trainer


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained('facebook/mbart-large-50', src_lang="bn_IN", tgt_lang="bn_IN", use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50")
model=model.to(device)
print(tokenizer.tokenize("উজ্জল, বৈশাখ, আক্তার"))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

['▁উ', 'জ', '্জ', 'ল', ',', '▁বৈ', 'শা', 'খ', ',', '▁আ', 'ক্ত', 'ার']


In [21]:
max_token_length = 32
trainer =setup(train, val, model, tokenizer, max_token_length, 1356300, 100, 1) #, 1356300, 56, 1)

Number of parameters: 610,879,488


In [22]:
i=38
print(tokenizer.decode(trainer.train_dataset[201]['input_ids']))
print(tokenizer.decode(trainer.train_dataset[201]['labels']))

bn_IN দুই মিনর সৈরঘ্জের চলচচিত্রটি ে নম আঈ েঅাম দা ফরেস্ট ।</s><pad>
bn_IN দুই মিনিট দৈর্ঘ্যের চলচ্চিত্রটির নাম আই অ্যাম দ্য ফরেস্ট ।</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [23]:
print(model.generation_config)

generation_config = GenerationConfig(
    max_length = max_token_length,
    num_beams = 5,
    early_stopping = True,
    forced_eos_token_id = 2,
    pad_token_id = 1,
    bos_token_id = 0,
    decoder_start_token_id = 2,

)

GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 5,
  "pad_token_id": 1
}



In [24]:
def show_training_duration(time1, time2):
    time_diff= time2-time1
    hours = time_diff // 3600
    minutes = (time_diff % 3600) // 60
    print(f"Time took to train: {int(hours)} hours and {int(minutes)} minutes")

In [25]:
from google.colab import runtime
time1=time.time()
try:
    try:
        try:
            last_checkpoint = get_last_checkpoint(output_dir)
            trainer.train(resume_from_checkpoint=last_checkpoint)
        except:
            trainer.train()

        time2=time.time()
        show_training_duration(time1, time2)

        generation_config.save_pretrained("/content/drive/MyDrive/Colab Notebooks/mBART 50/Models")
        trainer.save_model("/content/drive/MyDrive/Colab Notebooks/mBART 50/Models")

        generation_config.save_pretrained(output_dir+"models")
        trainer.save_model(output_dir+"models")
        print('Hello from the other side!')
        # runtime.unassign()
    except Exception as e:
        generation_config.save_pretrained("/content/drive/MyDrive/Colab Notebooks/mBART 50/Models")
        trainer.save_model("/content/drive/MyDrive/Colab Notebooks/mBART 50/Models")
        generation_config.save_pretrained(output_dir+"models")
        trainer.save_model(output_dir+"models")
        print(f"At least, I can say that I've tried{e}")
        # runtime.unassign()
except Exception as e:
    print(f"To tell you I'm sorry for breaking your heart: {e}")
    # runtime.unassign()


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Wer,Cer,Bleu,Meteor,Gen Len
500,0.539500,0.493198,0.112000,0.030000,0.275000,0.156000,0.486000,0.716000,20.401000
1000,0.391800,0.331244,0.113000,0.030000,0.191000,0.104000,0.628000,0.807000,20.055000
1500,0.341500,0.272956,0.113000,0.030000,0.162000,0.087000,0.685000,0.838000,19.920000


Prediction: রোহিতও স্পিনারের বিপক্ষে ভালো স্পিনারদের খেলতে ভালোবাসেন ।
Label: রোহিতও স্পিনের বিপক্ষে ভালো স্পিনারদের খেলতে ভালোবাসেন ।


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


Prediction: দেশের জন্য প্রাণ দেওয়া রবিউলের স্ত্রী কি জীবনের চাকরিটি নিতে পারেন?
Label: দেশের জন্য প্রাণ দেওয়া রবিউলের স্ত্রী কি করুণার চাকরিটি নিতে পারেন?


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


...

Prediction: বক স্প্লিন্টেরগুলো বের করতে অস্ত্রোপচারের জন্য শরীর এখন পর্যাপ্ত সবল ।
Label: বাকি স্প্লিন্টারগুলো বের করতে অস্ত্রোপচারের জন্য শরীর এখন পর্যাপ্ত সবল নয় ।


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Time took to train: 3 hours and 55 minutes


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


Hello from the other side!


In [27]:
last_checkpoint = get_last_checkpoint(output_dir)
print(last_checkpoint)


/content/drive/MyDrive/Colab Notebooks/mBART 50/Models/checkpoint-6500


In [ ]:
# model_name ="facebook/mbart-large-50"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def evaluate_model():
    tokenizer = AutoTokenizer.from_pretrained('facebook/mbart-large-50', src_lang="bn_IN", tgt_lang="bn_IN", use_fast=True)
    last_checkpoint = get_last_checkpoint(output_dir)
    print(last_checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/Colab Notebooks/mBART 50/Modelsmodels') #It will be different each time.
    model=model.to(device)

    print(f"Number of parameters: {model.num_parameters():,}")

    def correct_grammar(text, model, tokenizer):
        model=model.to(device)
        input_text = text
        inputs = tokenizer.encode(
            input_text,
            return_tensors='pt',
            max_length=len(text)+2,
        ).to(device)

        corrected_ids = model.generate(
            inputs,
            max_new_tokens=len(text)+2,
            num_beams=4,
            early_stopping=True
        ).to(device)

        # Decode.
        corrected_sentence = tokenizer.decode(
            corrected_ids[0],
            skip_special_tokens=True
        )
        return corrected_sentence

    predictions=[]
    references =[]
    for data in val_pairs: #[:10]:
        query = data[0]
        result = correct_grammar(query, model, tokenizer)
        predictions.append(result)
        references.append(data[1])
        print(f'Corrected query: {result}')

    wer = jiwer.wer(predictions, references)
    print(f'WER: {wer}')
    cer = jiwer.cer(predictions, references)
    print(f'CER: {cer}')
    def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [[label.strip()] for label in labels]
        return preds, labels

    decoded_preds, decoded_labels = postprocess_text(predictions, references)
    bleu_score = metric.compute(predictions=decoded_preds, references=decoded_labels)
    print(f"BLEU score: {bleu_score['score']/100}")
    meteor_score = meteor.compute(predictions=predictions, references=references)
    print(f"Meteor score: {meteor_score['meteor']}")

    spellchecker = pipeline('text2text-generation', model=model,
                            tokenizer=tokenizer, max_length=20)
    spellchecker = pipeline('text2text-generation', model=model,
                            tokenizer=tokenizer, max_length=20)
    while True:
        query = input("Search query:")
        result = spellchecker(query)
        print(f'Corrected query: {result[0]["generated_text"]}')

evaluate_model()

In [ ]:
import zipfile
import os
from IPython.display import FileLink
last_checkpoint = get_last_checkpoint(output_dir)
def zip_dir(directory = os.curdir, file_name = 'model.zip'):
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))
    return FileLink(file_name)
zip_dir(last_checkpoint)

In [ ]:
from google.colab import files
from datetime import datetime

def zip_dir(directory=os.curdir, base_filename='model_'):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    file_name = f'{base_filename}{timestamp}.zip'
    zip_path = os.path.join(directory, file_name)

    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(root, file)
                if file_path != zip_path:
                    zipf.write(file_path, os.path.relpath(file_path, start=directory))
    return zip_path

try:
  last_checkpoint = get_last_checkpoint(output_dir)
except:
  last_checkpoint = "/content/models"
zip_path = zip_dir(last_checkpoint)
files.download(zip_path)